# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

For brevity shorter alias names are used for the two metrics: `ARI` for [`adjusted_rand_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html) and `SIL` for [`silhouette_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html). Also `HC` alias is used for [`AgglomerativeClustering`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html).

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, matplotlib.pyplot as plt 
from sklearn.cluster import AgglomerativeClustering as HC  # hierarchical clustering
from sklearn.metrics import adjusted_rand_score as ARI, silhouette_score as SIL
from sentence_transformers import SentenceTransformer  # encodes text documents to 768D vectors
pd.set_option('max_rows', 5, 'max_columns', 40, 'max_colwidth', 100, 'precision', 2) # print format

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

Review the code Professor Melnikov used to subsample movies from The Movie Database ([TMDB](https://www.themoviedb.org/)) file `movies.zip`, encode their text attribute, cluster these movie vectors into two groups and compute several metrics of performance of this clustering mechanism.

Changes from the video:

1. As in the previous Jupyter Notebook (JN), the movies are filtered before they are encoded to avoid unnecessary encoding of unused movie descriptions
1. A smaller (in file size) pre-trained [SBERT](https://www.sbert.net/) model is used
1. Movie filtering is done simpler - based on language in the movie
1. For simplicity, a single textual attribute is encoded
1. Other minor code improvements are introduced

## **Read Movie Attributes**

The next cell reads the movie file, replaces missing (i.e., [NA](https://pandas.pydata.org/docs/user_guide/missing_data.html#missing-data-na)) values, sets movie titles as a row indices for each movie, and filters out all but two mutually exclusive languages, Spanish (with 32 movies) and German (with 27 movies). 



In [ ]:
df = pd.read_csv('movies.zip').fillna('').set_index('original_title') # load TMDB database
df.original_language.value_counts().to_frame().T
df = df.query('original_language=="es" or original_language=="de"')   # disjoint languages 
print('df.shape = ', df.shape)
df[:1]

The movie descriptions are encoded into a 768-dimensional space, where each movie is represented by a 768D numeric vector. Now mathematical calculations of distances (used in clustering) can be applied to any pair of vectors.

In [ ]:
SBERT = SentenceTransformer('paraphrase-albert-small-v2')  # load a pre-trained language model
mEmb  = SBERT.encode(df.overview)  # embedding ~5K descriptions may take 10+ minutes
pd.DataFrame(mEmb, index=df.title) # wrap matrix as dataframe with movie titles as indices

Next, a hierarchical clustering model is initialized and fitted on an embedding matrix of selected movies, `mEmb`, containing 59 rows as movie vectors.

In [ ]:
hc = HC(n_clusters=2).fit(mEmb)   # initialize and fit a clustering object on movie vectors

The attribute `hc.labels_` contains predicted cluster labels, which are always whole numbers starting from 0. These are assigned arbitrarily. So, one cluster can be assigned a label 0 for one movie sample and a label 1 for even a slightly different movie sample. This instability in label representation presents a problem if we want to compare estimated clusters to possibly known classes (if true labels are given in advance). 

Below several clustering performance measures are presented for situations when true labels are known and not known. Remember that clustering is an unsupervised learning method and does not use labels in identifying "similar" observations.

First, `tY` is defined as a vector of labels, 0 for German, 1 for Spanish. This assignment is arbitrary.

In [ ]:
pY = hc.labels_   # predicted labels, i.e. numbers 0,1 for estimated clusters
tY = (df.original_language=='es')*1  # true (observed) labels, 0=German, 1=Spanish
pd.DataFrame({'predicted labels, pY':pY, 'true labels, tY': tY}).T # compare labels

## **Accuracy Score**

Accuracy, a number in the $[0,1]$ range, is computed for approximately balanced (i.e. equisized) clusters. It is simply a fraction of matched labels. Since label assignment is arbitrary, you can compute accuracy for `pY == tY` and for `pY != tY`, and just use the largest of the two. If you have more than two clusters, you will need to try all combinations of matched estimated and true labels before you choose the greatest computed accuracy as the actual measure of performance. The complementary fraction becomes the error rate.

In [ ]:
Equal_rate = sum(pY == tY) / len(tY)
Unequal_rate = sum(pY != tY) / len(tY) # same as 1 - Equal_rate
print(f'Accuracy for pY == tY : {max(Equal_rate, Unequal_rate):.3f}')   # largest ratio is the accuracy rate
print(f'Error rate: {min(Equal_rate, Unequal_rate):.3f}') # Smallest ratio is the error rate

The accuracy of 0.78 (highest of the two) is hard to judge unless you are well familiar with this domain. Still the clustering appears effective (and not random), especially, since are are essentially identifying the language of each movie based on the vector representation of its English-written *overview* text. If changes are made to the model, it becomes easier to judge the change in accuracy (and not its absolute value). Higher accuracy tends to be more desirable.

## **Adjusted Rand Index**

The [Rand index](https://scikit-learn.org/stable/modules/clustering.html#rand-index) (RI), developed by statistician William Rand, counts all *pairs of points assigned to the same clusters* and those assigned otherwise. RI uses the true labels, which are not always available, but a human expert can also evaluate a smaller sample of paired points.

The [adjusted Rand index (ARI)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html#sklearn-metrics-adjusted-rand-score) rescales RI to be in [0,1] interval, where 0 indicates no pattern in clustering, and 1 is a perfect clustering with dense and well-separated groups. ARI is computed as

$$\frac{(\text{RI} - \text{RI}_{\text{Expected}})}{ (\max(\text{RI}) - \text{RI}_\text{Expected})} \in [0,1]$$

Next we compute ARI by providing true and estimated labels. While the metric of 0.30 appears rather low (compared to the value 1), the relative comparisons of this metric are more meaningful. Your goal should be to improve 0.30 value, which may be exponentially harder as it approaches the value 1. Typically, once you develop an expertise with a particular corpus, you will be able to assess whether 0.30 is too low or whether it's better than one might expect with the given corpus. The improvement in ARI is what you are after. Once it becomes too hard or expensive to improve (towards 1), perhaps, it's a good time to move to another task.

In [ ]:
ARI(tY, pY)   # ARI ∈ [0,1]

## **Silhouette Score (Coefficient)**

[Silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) relies on inter- and intra- cluster distances without observation labels. It is computed as 

$$(b - a) / \max(a, b) \in [-1,1]$$

where $a:=$ mean intra-cluster distance, $b:=$ mean nearest-cluster distance.

In [ ]:
SIL(mEmb, labels=pY)  # Silhouette Score, (b - a) / max(a, b) ∈ [-1,1]

## **Use Metrics To Estimate Number of Clusters**

Performance metrics are especially helpful when compared on a relative basis: you can choose a "better" number of clusters by selecting one with the best performance. For that, you need to try different numbers of clusters, $k$, say from two to 10 (and possibly larger for larger and more diverse sets). 

Below this is done with a function `ScoreClusters()`, which fits a hierarchical clustering model for the given $k$ and computes two metrics, ARI and silhouette score. Each $k$ and its corresponding measures are saved into a dataframe, which is plotted (metric vs $k$). 

In [ ]:
def ScoreClusters(k=2, tY=[0,1,1], mEmb=mEmb)->(float, float):
    '''Inputs:
        k: number of clusters
        tY: array/list of test/true labels
        mEmb: embedding matrix with rows as sentence vectors'''
    hc = HC(n_clusters=k).fit(mEmb)  # run clustering algorithm
    return ARI(np.array(tY), hc.labels_), SIL(mEmb, labels=hc.labels_)

K = range(2, 10)  # range of numbers of clusters
dfSC = pd.DataFrame([ScoreClusters(k, pY, mEmb) for k in K], index=K, columns=['ARI','SIL'])
dfSC.index.name='Number of clusters, k'
ax = dfSC.plot(grid=True, figsize=[15,4], secondary_y='SIL', title='Clustering Performance @k');

Both metrics suggest that two is the best number of clusters (shows highest ARI and silhouette scores at $k=2$). This makes sense, since we picked movies that mainly differ in language attribute.

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now you will practice writing a function to compute accuracy scores to measure cluster performance and determine the ideal number of clusters.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the **See solution** drop-down to view the answer.

## **Task 1**

Write a UDF `Acc(pY, tY)->float` which takes two arrays (binary predicted labels and binary true labels) and computes accuracy as the best accuracy from each combination of true label assignments (as discussed above).

<b>Hint:</b> You need to try every possible assignment of true labels, <code>tY</code>, and compute the corresponding accuracies. Then return the maximum. For the binary labels there are only two permutations of true labels.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
def Acc(pY=np.array([0,1,1]), tY=[0,0,1]) -> float:
    return max(sum(pY == tY), sum(pY != tY)) / len(tY)
Acc(pY, tY)
</pre>
</details> 
</font>

<hr>

## **Task 2**

Write a UDF `ScoreClusters2(k=2, tY, mEmb)->(float, float, float)`, which is similar to `ScoreClusters()`, but returns the accuracy score as well. Then run this UDF for $k\in\{2,3,...,30\}$. You can print the results on screen; plotting is not necessary.

What $k$ values of ARI, silhouette score, and accuracy imply the "best" clustering? Which one $k$ value would you choose using accuracy?

<b>Hint:</b> You can add the output of <code>Acc()</code> UDF as an additional value of the tuple returned by <code>ScoreClusters2()</code>

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
def ScoreClusters2(k=2, tY=[0,1,1], mEmb=mEmb)->(float, float, float):
    '''Inputs:
        k: number of clusters
        tY: array/list of test/true labels
        mEmb: embedding matrix with rows as sentence vectors'''
    hc = HC(n_clusters=k).fit(mEmb)  # run clustering algorithm
    return ARI(np.array(tY), hc.labels_), SIL(mEmb, labels=hc.labels_), Acc(hc.labels_, tY)

K = range(2, 30)  # range of numbers of clusters
dfSC2 = pd.DataFrame([ScoreClusters2(k, pY, mEmb) for k in K], index=K, columns=['ARI','SIL','ACC'])

#- print k maximizers (i.e. k values yielding highest values of the metrics)
print([list(dfSC2[m][dfSC2[m]==dfSC2[m].max()].index) for m in ['ARI','SIL','ACC']])

#- plot performance metrics at each value of k
ax = dfSC2.plot(grid=True, figsize=[15,4], secondary_y='SIL');
plt.title('Clustering Performance at each k');
ax.set_xlabel('number of clusters, k');
            </pre>ARI suggests $k=2$, SIL suggests $k=21$, accuracy suggests $k\in\{2, 3, 6, 8, 11, 16, 18, 23\}$. Typically, a "simpler" (or lower complexity) model should be preferred (for many reasons). Thus, given multiple $k$, we prefer the fewest number of clusters. Thus, $k=2$ is preferred using accuracy metric.
</details>
</font>

<hr>